### 보일러 시공서류 OCR 테스트 - 현대해상
- 작성일 : 2025. 4/30 

In [1]:
import fitz  
import os
import pandas as pd
import json
import requests
import uuid
import time

In [2]:
# 1. raw 폴더의 PDF 파일을, 페이지당 png파일로 변환하여 scan폴더에 저장 

def pdf_to_images(pdf_path, output_folder):
    pdf = fitz.open(pdf_path)
    for i in range(len(pdf)):
        page = pdf[i]
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
        pix.save(f"{output_folder}/page_{i+1:03d}.png")
    pdf.close()

if __name__ == "__main__":
    pdf_to_images("raw/현대해상.pdf", "scan")

In [3]:
# 2. Naver OCR을 이용하여, 텍스트 추출 

# NCP OCR API 설정
secret_key = 'ckliZ1FMSmtDcHd4elV4am52V2lzQnRQRm1rQlNZTHA='
api_url = 'https://aoomy83n4j.apigw.ntruss.com/custom/v1/41507/033430874e1f82158dc3c6505e0619708f396d6478568d229ce071464b5107f9/infer'

# 스캔 이미지가 저장된 폴더 지정
scan_folder = 'scan/'

# 각 파일의 텍스트 데이터를 저장할 딕셔너리
extracted_data = {}

# 폴더 내 모든 이미지 처리
for img_file in os.listdir(scan_folder):
    if img_file.endswith('.png'):
        img_path = os.path.join(scan_folder, img_file)
        filename = os.path.splitext(img_file)[0]  # 확장자 제거하여 파일명 추출
        
        # OCR API 호출
        files = [('file', open(img_path, 'rb'))]
        request_json = {
            'images': [
                {
                    'format': 'png',
                    'name': img_file,
                    'templateIds': [37217]
                }
            ],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000))
        }
        
        payload = {'message': json.dumps(request_json).encode('UTF-8')}
        headers = {'X-OCR-SECRET': secret_key}

        response = requests.request("POST", api_url, headers=headers, data=payload, files=files)
        result = response.json()

        # 각 파일별 텍스트 저장
        #extracted_data[filename] = [field['inferText'] for field in result['images'][0]['fields']]

In [4]:
result

{'version': 'V2',
 'requestId': '154f738e-0d0c-4e1e-89d3-045195faa02d',
 'timestamp': 1745979748692,
 'images': [{'uid': 'd5afd8dc8f2044f3b499833931a8eea3',
   'name': 'page_001.png',
   'inferResult': 'SUCCESS',
   'message': 'SUCCESS',
   'matchedTemplate': {'id': 37217, 'name': 'boiler_test_hd'},
   'validationResult': {'result': 'NO_REQUESTED'},
   'convertedImageInfo': {'width': 2480,
    'height': 3507,
    'pageIndex': 0,
    'longImage': False},
   'fields': [{'name': 'name',
     'valueType': 'ALL',
     'boundingPoly': {'vertices': [{'x': 719.0, 'y': 540.0},
       {'x': 880.0, 'y': 549.0},
       {'x': 876.0, 'y': 627.0},
       {'x': 714.0, 'y': 618.0}]},
     'inferText': '서하늘',
     'inferConfidence': 0.9987,
     'type': 'NORMAL',
     'subFields': [{'boundingPoly': {'vertices': [{'x': 719.0, 'y': 540.0},
         {'x': 880.0, 'y': 549.0},
         {'x': 876.0, 'y': 627.0},
         {'x': 714.0, 'y': 618.0}]},
       'inferText': '서하늘',
       'inferConfidence': 0.9987,


In [5]:
# 3. 추출된 텍스트를 하나의 엑셀 파일에 결합하여 저장 
import csv

# 파싱된 정보
data = {
    "name": "서하늘",
    "tel_no": "010-9270- 0810",
    "address": "평택시 안중읍 현덕호 7 신청아파트 103동 1503호",
    "maker_no": "C41A211510125\n016470924",
    "model_nm": "트윈알파 ECOL11- 168HEN",
    "heat_power": "16.000 Kcal/h",
    "set_date": "2024 년 10 월 28 일"
}


# CSV로 저장
csv_filename = "boiler_info_hd.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=data.keys())
    writer.writeheader()
    writer.writerow(data)

print(f"CSV 파일로 저장 완료: {csv_filename}")


CSV 파일로 저장 완료: boiler_info_hd.csv
